In [1]:
import numpy as np
import pandas as pd

In [2]:
stai_scores = pd.read_csv(f'./data_2023/stai_scores.csv', header=None, names=['stai'])

# Task (a)

In [3]:
stai_mean = stai_scores.mean()

stai_std = stai_scores.std()

stai_median = stai_scores.median()

In [4]:
f"stai mean: {float(stai_mean)}"

'stai mean: 42.72'

In [5]:
f"stai std: {float(stai_std)}"

'stai std: 14.880531041654146'

In [6]:
f"stai median: {float(stai_median)}"

'stai median: 40.0'

In [7]:
cutoff = 43

illnes = stai_scores[stai_scores.stai >= cutoff]

illnes

,stai
0,65
1,46
2,61
4,51
5,48
6,65
7,47
8,63
9,57
10,73


In [8]:
list(illnes.index)

[0, 1, 2, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 16, 17, 18, 19, 20, 21, 22, 23]

In [9]:
[i for i in range(26) if i not in list(illnes.index)]

[3, 15, 24, 25]

In [10]:
inst_choices = pd.read_csv(f'./data_2023/inst_choices.csv', header=None)

In [11]:
choice_count = inst_choices.apply(lambda x: x.value_counts(), axis=1)

A_count = choice_count[1]
B_count = choice_count[2]

In [12]:
A_count.mean()

34.46

In [13]:
choice_count.iloc[2]

2    128
1     32
Name: 2, dtype: int64

# Task (b)

In [14]:
def V(v_t, a, o):
    return v_t + a * (o - v_t)

In [15]:
def p(v_a, v_b, b):
    return np.exp((-b) * v_a) / ((np.exp((-b) * v_a)) + (np.exp((-b) * v_b)))

In [16]:
def change_state(last_state):
    if last_state == (0.7, 0.3):
        return (0.8, 0.2)
    elif last_state == (0.8, 0.2):
        return (0.6, 0.4)
    elif last_state == (0.6, 0.4):
        return (0.65, 0.35)
    elif last_state == (0.65, 0.35):
        return (0.7, 0.3)
    else:
        raise ValueError(f'Unexpected {last_state}')

In [17]:
alpha = 0.4
beta = 7

In [18]:
def generate_stimuli():
    state = (0.7, 0.3)

    num = 160

    sim_choice = np.zeros(num)
    sim_outcome = np.zeros(num)

    V_A = 0.5
    V_B = 0.5

    for t in range(num):
        if (t + 1) % 40 == 0:
            state = change_state(state)

        p_A = p(V_A, V_B, beta)

        choice = 1 if np.random.rand() < p_A else 2

        check = np.random.rand()
        thres = state[0]

        outcome = int(check < thres) if choice == 1 else int(check > thres)

    #     print(f"c: {'A' if choice == 1 else 'B'}, o: {'-' if outcome == 0 else 'aversive'}")

        sim_choice[t] = choice
        sim_outcome[t] = outcome

        V_A = V(V_A, alpha, outcome) if choice == 1 else V_A
        V_B = V(V_B, alpha, outcome) if choice == 2 else V_B
    
    return sim_outcome

In [20]:
avg_aversive = 0

for i in range(0, 500):
    avg_aversive = (avg_aversive + generate_stimuli()[0: 160].sum()) / 2
    
avg_aversive

58.38223641464315